In [ ]:
!pip install requests
!pip install pymysql
!pip install pandas
!pip install mysql-connector-python

In [79]:
import requests
import json
import pymysql
import mysql.connector
import datetime

# **Data_Collection,Cleaning_and_Transformation**

In [80]:

API_KEY ="iybcULnPqJDgAcRaKA77cL9cA2zGtciiAuDEj5l6"
url = f"https://api.nasa.gov/neo/rest/v1/feed?start_date=2025-05-01&end_date=2025-05-08&api_key={API_KEY}"
target = 10000
ast_data =[]
count = 1
while len(ast_data)<target:
  response = requests.get(url)
  data = response.json()
  total_details =data['near_earth_objects']
  for date,ast_details in total_details.items():
    for ast in ast_details:
      ast_data.append(dict(
          serial_no = count,        #For_Reference
          id =int(ast['id']),
          name = ast['name'],
          absolute_magnitude_h = float(ast['absolute_magnitude_h']),
          estimated_diameter_min_km = float(ast['estimated_diameter']['kilometers']['estimated_diameter_min']),
          estimated_diameter_max_km = float(ast['estimated_diameter']['kilometers']['estimated_diameter_max']),
          is_potentially_hazardous_asteroid = ast['is_potentially_hazardous_asteroid'],
          neo_reference_id = int(ast['neo_reference_id']),
          close_approach_date =datetime.datetime.strptime(ast['close_approach_data'][0]['close_approach_date'], "%Y-%m-%d").date(),
          reltive_velocity_kmph = float(ast['close_approach_data'][0]['relative_velocity']['kilometers_per_hour']),
          astronomical = float(ast['close_approach_data'][0]['miss_distance']['astronomical']),
          miss_distance_km = float(ast['close_approach_data'][0]['miss_distance']['kilometers']),
          miss_distance_lunar = float(ast['close_approach_data'][0]['miss_distance']['lunar']),
          orbiting_body =ast['close_approach_data'][0]['orbiting_body']

                           )
      )

      count+=1
      if len(ast_data) == target:
        break
    if len(ast_data) == target:
        break
  url = data['links'].get('next')

In [ ]:
ast_data

In [81]:
import pandas as pd

df = pd.DataFrame(ast_data)

# Export to CSV
df.to_csv('ast_data.csv', index=False)

# Export to Excel
df.to_excel('ast_data.xlsx', index=False)

print("Exported to ast_data.csv and ast_data.xlsx")


Exported to ast_data.csv and ast_data.xlsx


# **SQL connection and DB & Table creation **

In [67]:
connection = mysql.connector.connect(                             #SQL_Connection
    host = "gateway01.eu-central-1.prod.aws.tidbcloud.com",
    user = "m5Xt4HUDrniTj1o.root",
    password= "JhWWCZgwpY56epBA",
   )
cur_sor = connection.cursor()

In [ ]:
cur_sor.execute("create database NEO")

In [68]:
cur_sor.execute("use NEO")

In [69]:
cur_sor = connection.cursor()

In [ ]:
cur_sor.execute("use NEO")
cur_sor.execute('''create table if not exists asteroids(
  id INT,
  name VARCHAR(255),
  absolute_magnitude_h FLOAT,
  estimated_diameter_min_km FLOAT,
  estimated_diameter_max_km FLOAT,
  is_potentially_hazardous_asteroid BOOLEAN
)''')
connection.commit()

In [ ]:
line = 1;
for ast in ast_data:
  print("Execution line ", line  )#To_Verify_that_lines_are_executing
  cur_sor.execute('''insert into asteroids(id,name,absolute_magnitude_h,estimated_diameter_min_km,estimated_diameter_max_km,is_potentially_hazardous_asteroid) values(%s,%s,%s,%s,%s,%s)''',(ast['id'],ast['name'],ast['absolute_magnitude_h'],ast['estimated_diameter_min_km'],ast['estimated_diameter_max_km'],ast['is_potentially_hazardous_asteroid']))
  line+=1
connection.commit()

In [ ]:
cur_sor.execute("use NEO")
cur_sor.execute('''create table if not exists close_approach
(
  neo_reference_id INT,
  close_approach_date DATE,
  relative_velocity_kmph FLOAT,
  astronomical FLOAT,
  miss_distance_km FLOAT,
  miss_distance_lunar FLOAT,
  orbiting_body VARCHAR(255)
)''')
connection.commit()

In [ ]:
line = 1;
for ast in ast_data:
  print("Execution line ", line  )
  cur_sor.execute('''insert into close_approach(neo_reference_id,close_approach_date,relative_velocity_kmph,astronomical,miss_distance_km,miss_distance_lunar,orbiting_body) values(%s,%s,%s,%s,%s,%s,%s)''',(ast['neo_reference_id'],ast['close_approach_date'],ast['reltive_velocity_kmph'],ast['astronomical'],ast['miss_distance_km'],ast['miss_distance_lunar'],ast['orbiting_body']))
  line+=1
connection.commit()

#**SQL_Queries**

In [ ]:
#Query_1
#Count how many times each asteroid has approached Earth
cur_sor.execute('''select neo_reference_id as AsteroidID ,count(*) as HowManyTimesApproached from close_approach group by neo_reference_id order by HowManyTimesApproached desc''')
cur_sor.fetchall()

In [ ]:
#Query_2
#Average velocity of each asteroid over multiple approaches
cur_sor.execute('''select neo_reference_id as AsteroidID,avg(relative_velocity_kmph) as AverageVelocityKmph from close_approach group by AsteroidID having count(AsteroidID)>1 order by AverageVelocityKmph desc ''')
cur_sor.fetchall()

In [ ]:
#Query_3
#List top 10 fastest asteroids
cur_sor.execute('''select neo_reference_id as AsteroidID,relative_velocity_kmph as Velocity_Kmph from close_approach order by Velocity_Kmph desc limit 10''')
cur_sor.fetchall()

In [ ]:
#Query_4
#Find potentially hazardous asteroids that have approached Earth more than 3 times
cur_sor.execute('''select asteroids.id as AsteroidID,count(close_approach.neo_reference_id) as ApproachCount from asteroids  join close_approach  on asteroids.id = close_approach.neo_reference_id where asteroids.is_potentially_hazardous_asteroid = TRUE group by asteroids.id having count(*)>3 order by ApproachCount desc''')
cur_sor.fetchall()

In [ ]:
#Query_5
#Find the month with the most asteroid approaches
cur_sor.execute('''select date_format(close_approach_date,'%Y-%m') as Month,count(*) as count from close_approach group by Month order by count Desc limit 1''')
cur_sor.fetchall()

In [ ]:
#Query_6
#Get the asteroid with the fastest ever approach speed
cur_sor.execute('''select asteroids.name,max(close_approach.relative_velocity_kmph) as SpeedinKmpH from asteroids join close_approach on asteroids.id = close_approach.neo_reference_id group by asteroids.name order by SpeedinKmpH desc''')
cur_sor.fetchall()

In [ ]:
#Query_7
#Sort asteroids by maximum estimated diameter (descending)
cur_sor.execute('''select name,estimated_diameter_max_km from asteroids group by name,estimated_diameter_max_km order by estimated_diameter_max_km desc''')
cur_sor.fetchall()

In [ ]:
#Query_8
#An asteroid whose closest approach is getting nearer over time
cur_sor.execute('''select asteroids.name,close_approach.miss_distance_km,close_approach.close_approach_date from asteroids join close_approach on asteroids.id = close_approach.neo_reference_id order by close_approach.miss_distance_km, close_approach.close_approach_date desc limit 1  ''')
cur_sor.fetchall()

In [ ]:
#Query_9
#Display the name of each asteroid along with the date and miss distance of its closest approach to Earth.

cur_sor.execute('''select asteroids.name,close_approach.miss_distance_km,close_approach.close_approach_date from asteroids join close_approach on asteroids.id = close_approach.neo_reference_id order by close_approach.miss_distance_km, close_approach.close_approach_date desc  ''')
cur_sor.fetchall()

In [ ]:
#Query_10
#List names of asteroids that approached Earth with velocity > 50,000 km/h
cur_sor.execute('''select asteroids.name from asteroids join close_approach on asteroids.id = close_approach.neo_reference_id where (close_approach.relative_velocity_kmph > 50000) order by asteroids.name  desc  ''')
cur_sor.fetchall()

In [ ]:
#Query_11
#Count how many approaches happened per month
cur_sor.execute('''select date_format(close_approach.close_approach_date,'%Y-%m') as Month,count(*) as count from close_approach group by Month order by Month asc  ''')
cur_sor.fetchall()

In [ ]:
#Query_12
#Find asteroid with the highest brightness (lowest magnitude value)
cur_sor.execute('''select name,id,absolute_magnitude_h from asteroids where absolute_magnitude_h=(select min(absolute_magnitude_h) from asteroids)''')
cur_sor.fetchall()

In [ ]:
#Query_13
#Get number of hazardous vs non-hazardous asteroids
cur_sor.execute('''select sum(is_potentially_hazardous_asteroid=true)as Hazard,sum(is_potentially_hazardous_asteroid=false) as NonHazard from asteroids ''')
cur_sor.fetchall()

In [ ]:
#Query_14
#Find asteroids that passed closer than the Moon (lesser than 1 LD), along with their close approach date and distance.
cur_sor.execute('''select asteroids.nameas Name_of_Asteroid,close_approach.miss_distance_lunar,close_approach.close_approach_date from asteroids join close_approach on asteroids.id = close_approach.neo_reference_id where (close_approach.miss_distance_lunar < 1) order by close_approach.miss_distance_lunar desc  ''')
cur_sor.fetchall()


In [ ]:
#Query_15
#Find asteroids that came within 0.05 AU(astronomical distance)
cur_sor.execute('''select asteroids.name as Name_of_Asteroid,close_approach.astronomical from asteroids join close_approach on asteroids.id = close_approach.neo_reference_id where (close_approach.astronomical < 0.05) order by close_approach.astronomical desc  ''')
cur_sor.fetchall()

In [ ]:
#Query_16
# Average estimated diameter of all asteroids
cur_sor.execute('''select avg(estimated_diameter_max_km) from asteroids''')
cur_sor.fetchall()

In [71]:
#Query_17
#Find the asteroid with the most recorded approaches
cur_sor.execute('''select asteroids.name as Name_of_Asteroid,count(close_approach.neo_reference_id) as No_of_times_approached from asteroids join close_approach on asteroids.id = close_approach.neo_reference_id group by asteroids.name order by count(close_approach.neo_reference_id) desc limit 1''')
cur_sor.fetchall()

[('(2023 QB2)', 64)]

In [ ]:
#Query_18
#Find asteroids that have approached Earth more than 1 year
cur_sor.execute('''select asteroids.name as Name_of_Asteroid,count(distinct year(close_approach.close_approach_date)) as No_of_Years_approached from asteroids join close_approach on asteroids.id = close_approach.neo_reference_id group by asteroids.name having count(distinct year(close_approach.close_approach_date))>1 order by No_of_Years_approached desc''')
cur_sor.fetchall()

In [ ]:
#Query_19
#Total number of asteroid approaches each year
cur_sor.execute('''select year(close_approach_date) as Year, count(*) as No_of_approches from close_approach group by year(close_approach_date) order by year(close_approach_date) asc''')
cur_sor.fetchall()

In [ ]:
#Query_20
# Find asteroids that have only approached Earth once
cur_sor.execute('''select asteroids.name as Name_of_Asteroid from asteroids join close_approach on asteroids.id = close_approach.neo_reference_id group by asteroids.name having count(distinct (close_approach.neo_reference_id))=1''')
cur_sor.fetchall()